DOCKER

Docker File

In [ ]:
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt /app/

RUN pip install --no-cache-dir -r requirements.txt

COPY . /app

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]


docker-compose.yml

In [ ]:
version: '3.8'

services:
  web:
    build: .
    ports:
      - "8000:8000"
    volumes:
      - .:/app
    environment:
      - CSV_FILE_PATH=/app/vehicles_data.csv


**Building and Running Docker Container**

In [ ]:
#building
docker build -t vms

#running
docker run -p 80:80 vms


# DEPLOYMENT Tools

Using Docker Swarm

In [ ]:
docker swarm init


Creating docker swarm service

In [ ]:
docker service create --name vehicle-management --publish 8000:8000 vehicle-management-system


Using Kubernetes

create deployment.yaml (Kubernetes deployment file)

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: vehicle-management
spec:
  replicas: 3
  selector:
    matchLabels:
      app: vehicle-management
  template:
    metadata:
      labels:
        app: vehicle-management
    spec:
      containers:
      - name: vehicle-management
        image: vehicle-management-system:latest
        ports:
        - containerPort: 8000


Deployment command:

In [ ]:
kubectl apply -f deployment.yaml


Using Heroku:

create a procfile

In [ ]:
web: uvicorn main:app --host 0.0.0.0 --port $PORT


Deployment commands:

In [ ]:
heroku create


In [ ]:
git push heroku main


# CLOUD Platforms

deployment in the cloud platforms also uses dockerfile

AWS

create dockerfile

In [ ]:
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt /app/

RUN pip install --no-cache-dir -r requirements.txt

COPY . /app

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]


Deployment Commands:

In [ ]:
#initialize elastic beanstalk application
eb init -p python-3.8 vms-app --region us-west-2


In [ ]:
#Create and Deploy the Environment
eb create vms-env
eb deploy


Azure

Create a Web App on Azure

In [ ]:
#Login to Azure CLI
az login

#Create Resource Group and App Service Plan
az group create --name vmsResourceGroup --location eastus
az appservice plan create --name vmsAppServicePlan --resource-group vmsResourceGroup --sku B1 --is-linux

#Create a Web App
az webapp create --resource-group vmsResourceGroup --plan vmsAppServicePlan --name vmsApp --runtime "PYTHON|3.8"

Deployment

In [ ]:
az webapp up --name vmsApp --resource-group vmsResourceGroup --plan vmsAppServicePlan


GOOGLE Cloud Platform GCP

Create App Engine Project


In [ ]:
#Initialize gcloud and set project
gcloud init

#Create App Engine Application
gcloud app create --project=vms-project --region=us-central


Deployment

create app engine configuration app.yaml

In [ ]:
runtime: python39
entrypoint: uvicorn main:app --host 0.0.0.0 --port $PORT

handlers:
- url: /.*
  script: auto


In [ ]:
#deploy
gcloud app deploy


# CI/CD Pipelines with GitHub Actions

CI/CD pipeline automates the process of testing, building, and deploying the application.

Create GitHub Actions Workflow

Create a ci-cd.yml file

In [ ]:
name: CI/CD Pipeline
#Triggering the Workflow
on:
  push:
    branches:
      - main
#Build Job
jobs:
  build:
    runs-on: ubuntu-latest
    #Checks out the code from the repository
    steps:
    - name: Checkout code
      uses: actions/checkout@v2
    #Sets up Python 3.9 environment
    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: 3.9
    #Installs dependencies from requiremnt.txt
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
    #Executes tests using pytest
    - name: Test with pytest
      run: |
        pytest
    #Builds Docker image
    - name: Build Docker image
      run: |
        docker build -t vehicle-management-system .
    #Logs in to Docker Hub
    - name: Log in to Docker Hub
      run: echo "${{ secrets.DOCKER_PASSWORD }}" | docker login -u "${{ secrets.DOCKER_USERNAME }}" --password-stdin
    #Pushes Docker image to Docker Hub
    - name: Push Docker image
      run: |
        docker tag vehicle-management-system:latest yourdockerhubusername/vehicle-management-system:latest
        docker push yourdockerhubusername/vehicle-management-system:latest

#Deploys the job
  deploy:
    needs: build
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v2

    - name: Deploy to AWS Elastic Beanstalk
      env:
        AWS_ACCESS_KEY_ID: ${{ secrets.AWS_ACCESS_KEY_ID }}
        AWS_SECRET_ACCESS_KEY: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
        AWS_DEFAULT_REGION: 'us-west-2'
      run: |
        pip install awsebcli
        eb init -p python-3.8 vehicle-management -r us-west-2
        eb deploy #Deploys to AWS Elastic Beanstalk
